In [ ]:
# Performing joins and cleaning transaction data 

In [130]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import math

In [131]:
# Finding current directory within JupyterHub
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /datar10/jupyter/user/josh.hall/condaenvs/MYENV/DATA-793 Capstone Curbside/dc_data


In [132]:
# Setting working directory
direc_path = '/datar10/jupyter/user/josh.hall/condaenvs/MYENV/DATA-793 Capstone Curbside/dc_data'

os.chdir(direc_path)

In [133]:
# In the DDOT provided data, missing values are noted as "NULL"
#Replacing "NULL" with "NaN" to avoid syntax errors in Pandas
def renaming_nulls(df):
    return df.replace("NULL", np.nan)

In [68]:
# Loading in parquet files from `01_data_querying.ipynb`
dc_trans_18 = pd.read_parquet('dc_2018_parking_payments.parquet')

In [69]:
dc_trans_18

,meternumber,spacenumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
0,NULL,NULL,NULL,2018-01-01 00:04:36,2018-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,2298a7fa-ee61-4351-bff9-3c52730f35b0
1,NULL,NULL,NULL,2018-01-01 00:06:09,2018-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,1785a6f5-ab67-4bcc-a5ef-5a8fac701053
2,NULL,NULL,NULL,2018-01-01 00:06:31,2018-01-01 01:06:31,0.00,PayByCell,PARKMOBILE,da920775-a7f9-4fd9-8a0d-417c8be3eb57
3,NULL,NULL,NULL,2018-01-01 00:10:14,2018-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,29dff519-7f21-48b4-9fcb-4f0d62bc5d19
4,NULL,NULL,NULL,2018-01-01 00:42:11,2018-01-01 01:42:11,0.00,PayByCell,PARKMOBILE,ecd65e6a-5943-4cc3-a7f9-92ddca52a7e5
...,...,...,...,...,...,...,...,...,...
15083827,35363406,35363406,NULL,2018-12-30 23:40:57,2018-12-30 23:40:57,0.25,Cash,IPS,18253165
15083828,35363406,35363406,NULL,2018-12-30 23:41:00,2018-12-30 23:41:00,0.25,Cash,IPS,18253166
15083829,35363406,35363406,NULL,2018-12-30 23:41:02,2018-12-30 23:41:02,0.25,Cash,IPS,18253167
15083830,35363406,35363406,NULL,2018-12-30 23:41:04,2018-12-30 23:41:04,0.50,Cash,IPS,18253168


In [71]:
# Cleaning and removing duplicates and transactions that lack enough information for analysis
dc_18 = renaming_nulls(dc_trans_18)
dc_18 = dc_18.dropna(subset=['meternumber', 'zone'], how='all') # need at least one
dc_18 = dc_18.drop(dc_18[dc_18['paymentamount'] == 0.00].index)
dc_18 = dc_18.drop_duplicates()

In [72]:
dc_18

,meternumber,spacenumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
5,20170908,20170908,NaN,2018-01-01 00:52:10,2018-01-01 00:52:10,0.35,Cash,IPS,13996795
6,21132320,21132320,NaN,2018-01-01 01:01:14,2018-01-01 01:01:14,0.50,Cash,IPS,13996796
8,10142111,10142111,NaN,2018-01-01 01:22:20,2018-01-01 01:22:20,0.25,Cash,IPS,13996803
10,11211307,11211307,NaN,2018-01-01 03:45:48,2018-01-01 03:45:48,1.00,Cash,IPS,13996830
11,21121814,21121814,NaN,2018-01-01 03:50:36,2018-01-01 03:50:36,0.25,Cash,IPS,14029819
...,...,...,...,...,...,...,...,...,...
15083827,35363406,35363406,NaN,2018-12-30 23:40:57,2018-12-30 23:40:57,0.25,Cash,IPS,18253165
15083828,35363406,35363406,NaN,2018-12-30 23:41:00,2018-12-30 23:41:00,0.25,Cash,IPS,18253166
15083829,35363406,35363406,NaN,2018-12-30 23:41:02,2018-12-30 23:41:02,0.25,Cash,IPS,18253167
15083830,35363406,35363406,NaN,2018-12-30 23:41:04,2018-12-30 23:41:04,0.50,Cash,IPS,18253168


In [73]:
# Checking null counts
dc_18.isna().sum()

meternumber                0
spacenumber                0
zone                 7676023
paymentstart               0
paymentexpiry              0
paymentamount              0
paymenttype                0
transactionsender          0
transmissionid             0
dtype: int64

In [ ]:
# Zone missing for every transaction from 2018

In [74]:
# Loading in parquet files from `01_data_querying.ipynb`
dc_trans_19 = pd.read_parquet('dc_2019_parking_payments.parquet')

In [75]:
dc_trans_19

,meternumber,spacenumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
0,NULL,NULL,NULL,2019-02-13 08:07:07,2019-02-13 09:07:07,2.30,PayByCell,PARKMOBILE,e5a64774-2ffe-41df-bf93-9c267ec6cbc5
1,61430592,PayAndDisplay,NULL,2019-02-13 08:07:10,2019-02-13 10:07:10,4.60,CreditCard,Parkeon,1550063316067824500
2,NULL,NULL,NULL,2019-02-13 08:07:12,2019-02-13 12:07:12,9.20,PayByCell,PARKMOBILE,79fe8ee5-55c1-4cc9-8dda-c39831c1e685
3,NULL,NULL,NULL,2019-02-13 08:07:12,2019-02-13 12:07:12,9.20,PayByCell,PARKMOBILE,17c06241-01b6-427b-9d1b-4e96f1b63b7e
4,NULL,NULL,NULL,2019-02-13 08:07:17,2019-02-13 09:07:17,4.75,PayByCell,PARKMOBILE,0cb72746-e00d-4393-8ec6-2ea6c51a3f82
...,...,...,...,...,...,...,...,...,...
14528341,NULL,NULL,NULL,2019-12-30 23:16:30,2019-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,77de26d9-34be-4e2a-b625-c6a5a95574d8
14528342,NULL,NULL,NULL,2019-12-30 23:32:55,2019-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,1412fba9-f484-4bca-a6c8-e89399798a55
14528343,NULL,NULL,NULL,2019-12-30 23:43:38,2019-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,8cb730a9-e457-434d-8b11-8a0dcf4feabf
14528344,NULL,NULL,NULL,2019-12-30 23:43:58,2019-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,82cc1f52-8749-4f09-9072-ce5b93c3d1b3


In [76]:
# Cleaning and removing duplicates and transactions that lack enough information for analysis
dc_19 = renaming_nulls(dc_trans_19)
dc_19 = dc_19.dropna(subset=['meternumber', 'zone'], how='all')
dc_19 = dc_19.drop(dc_19[dc_19['paymentamount'] == 0.00].index)
dc_19 = dc_19.drop_duplicates()

In [77]:
dc_19

,meternumber,spacenumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
1,61430592,PayAndDisplay,NaN,2019-02-13 08:07:10,2019-02-13 10:07:10,4.60,CreditCard,Parkeon,1550063316067824500
8,11201393,PayAndDisplay,NaN,2019-02-13 08:07:30,2019-02-13 08:21:50,0.55,Cash,Parkeon,1550063272060515000
9,25023218,25023218,NaN,2019-02-13 08:07:34,2019-02-13 08:52:43,1.00,Cash,IPS,18626543
14,61080092,PayAndDisplay,NaN,2019-02-13 08:07:40,2019-02-13 10:07:40,4.60,CreditCard,Parkeon,1550063238004636400
15,20211501,20211501,NaN,2019-02-13 08:07:41,2019-02-13 08:15:31,0.30,Cash,IPS,18625293
...,...,...,...,...,...,...,...,...,...
14528293,21071491,PayAndDisplay,NaN,2019-12-30 21:57:18,2019-12-30 22:00:00,0.10,Cash,Parkeon,1577761024038773000
14528296,61540192,PayAndDisplay,NaN,2019-12-30 21:59:44,2019-12-30 22:00:00,0.25,CreditCard,Parkeon,1577761320086554800
14528307,21092203,21092203,NaN,2019-12-30 22:05:52,2019-12-30 22:05:52,0.25,Cash,IPS,21969105
14528308,21092203,21092203,NaN,2019-12-30 22:05:55,2019-12-30 22:05:55,0.25,Cash,IPS,21969106


In [78]:
# Checking null counts
dc_19.isna().sum()

meternumber                0
spacenumber                0
zone                 6429062
paymentstart               0
paymentexpiry              0
paymentamount              0
paymenttype                0
transactionsender          0
transmissionid             0
dtype: int64

In [134]:
# Loading in parquet files from `01_data_querying.ipynb`
dc_trans_20 = pd.read_parquet('dc_2020_parking_payments.parquet')

In [135]:
dc_trans_20

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
0,NULL,NULL,2020-01-01 00:01:44,2020-01-01 01:02:44,0.00,PayByCell,PARKMOBILE,a65478c5-6358-4484-9c5e-30fb29a328fd
1,NULL,NULL,2020-01-01 00:03:12,2020-01-01 01:03:12,0.00,PayByCell,PARKMOBILE,bc7fd6f4-2bd0-4ed9-afa4-8c45bf3a67f7
2,NULL,NULL,2020-01-01 00:03:25,2020-01-01 04:03:25,0.00,PayByCell,PARKMOBILE,b4a41421-34d3-4239-9efd-c0b574cf9394
3,NULL,NULL,2020-01-01 00:03:28,2020-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,b9a71bad-a2af-463d-a8f9-c7c1d6047ecc
4,NULL,NULL,2020-01-01 00:07:35,2020-01-01 03:07:35,0.00,PayByCell,PARKMOBILE,fc689131-4b2c-415c-b995-4b18b3cfa40c
...,...,...,...,...,...,...,...,...
5924555,NULL,2298,2020-12-30 22:11:15,2020-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,54564b9f-4285-4199-8b1d-b91ce485d211
5924556,NULL,22880,2020-12-30 22:12:35,2020-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,53198dd5-8dbb-4a06-8169-fe03eef0ab9c
5924557,22071602,NULL,2020-12-30 22:44:39,2020-12-30 22:44:39,0.25,Cash,IPS,23405673
5924558,NULL,22792,2020-12-30 22:54:10,2020-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,748c37d9-fff2-4cd5-96b0-a8cb3997f3d6


In [136]:
dc_trans_20.groupby('paymenttype').count()

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,transactionsender,transmissionid
paymenttype,,,,,,,
Cash,991745,991745,991745,991745,991745,991745,991745
CreditCard,1407485,1407485,1407485,1407485,1407485,1407485,1407485
PayByCell,3525330,3525330,3525330,3525330,3525330,3525330,3525330


In [137]:
# 2020 null count to see how many instances we will lose due to lack of geo data 
# rather than a 0 payment or repeat data
dc_20 = renaming_nulls(dc_trans_20)
null_count_20 = dc_20[(dc_20['meternumber'].isnull()) & (dc_20['zone'].isnull())].shape[0]
null_count_20

3192809

In [138]:
# More cleaning to match tables of data from other years
dc_20 = dc_20.dropna(subset=['meternumber', 'zone'], how='all')
dc_20 = dc_20.drop(dc_20[dc_20['paymentamount'] == 0.00].index)
dc_20 = dc_20.drop_duplicates()

In [139]:
dc_20

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
11,20151005,NaN,2020-01-01 00:44:26,2020-01-01 00:44:26,1.75,Cash,IPS,21977521
13,44297809,NaN,2020-01-01 01:02:56,2020-01-01 01:02:56,0.35,Cash,IPS,21978945
14,44297809,NaN,2020-01-01 01:03:26,2020-01-01 01:03:26,0.50,Cash,IPS,21978946
19,20191507,NaN,2020-01-01 01:16:31,2020-01-01 01:16:31,0.25,Cash,IPS,21977786
25,20110601,NaN,2020-01-01 01:36:45,2020-01-01 01:36:45,0.25,Cash,IPS,21978904
...,...,...,...,...,...,...,...,...
5924548,10142191,NaN,2020-12-30 21:53:25,2020-12-30 22:00:00,0.25,CreditCard,Parkeon,1609383345032849400
5924549,20170420,NaN,2020-12-30 21:53:26,2020-12-30 21:59:58,0.25,Cash,IPS,23405162
5924550,10142191,NaN,2020-12-30 21:55:54,2020-12-30 22:00:00,0.25,CreditCard,Parkeon,1609383445046022700
5924551,10142191,NaN,2020-12-30 21:57:33,2020-12-30 22:00:00,0.25,CreditCard,Parkeon,1609383541022008000


In [140]:
# Null counts
dc_20.isna().sum()

meternumber           317779
zone                 2399119
paymentstart               0
paymentexpiry              0
paymentamount              0
paymenttype                0
transactionsender          0
transmissionid             0
dtype: int64

In [141]:
# Loading in parquet files from `01_data_querying.ipynb`
dc_trans_21 = pd.read_parquet('dc_2021_parking_payments.parquet')

In [142]:
dc_trans_21

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
0,NULL,21650,2021-01-01 00:00:33,2021-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,29415cee-58e6-40b1-974c-5d29c8685332
1,NULL,22059,2021-01-01 00:00:37,2021-01-01 01:30:37,0.00,PayByCell,PARKMOBILE,68734915-459c-4df1-b245-328d32d3e93b
2,NULL,22142,2021-01-01 00:01:41,2021-01-01 10:01:41,0.00,PayByCell,PARKMOBILE,6df16575-dbaa-4784-acad-3e885eb59ad4
3,NULL,22986,2021-01-01 00:03:33,2021-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,4d89bf08-563d-40a3-b85c-e6cca111688b
4,NULL,21621,2021-01-01 00:04:45,2021-01-01 01:04:45,0.00,PayByCell,PARKMOBILE,244df94a-a6ff-4205-ab1f-1ddd2f3772bf
...,...,...,...,...,...,...,...,...
6748138,12122808,NULL,2021-12-30 23:31:13,2021-12-30 23:31:13,0.25,Cash,IPS,24834152
6748139,12122808,NULL,2021-12-30 23:31:15,2021-12-30 23:31:15,0.25,Cash,IPS,24834154
6748140,12122808,NULL,2021-12-30 23:31:17,2021-12-30 23:31:17,0.25,Cash,IPS,24834155
6748141,NULL,21726,2021-12-30 23:33:29,2021-12-30 23:59:59,0.00,PayByCell,PARKMOBILE,e49b495b-ef85-42be-8971-904bb74949e5


In [143]:
# 2021 null count to see how many instances we will lose due to lack of geo data 
# rather than a 0 payment or repeat data
dc_21 = renaming_nulls(dc_trans_21)
null_count_21 = dc_21[(dc_21['meternumber'].isnull()) & (dc_21['zone'].isnull())].shape[0]
null_count_21

0

In [144]:
# More cleaning for uniformity
dc_21 = dc_21.dropna(subset=['meternumber', 'zone'], how='all')
dc_21 = dc_21.drop(dc_21[dc_21['paymentamount'] == 0.00].index)
dc_21 = dc_21.drop_duplicates()

In [145]:
# Null counts
dc_21.isna().sum()

meternumber          3970745
zone                 2596068
paymentstart               0
paymentexpiry              0
paymentamount              0
paymenttype                0
transactionsender          0
transmissionid             0
dtype: int64

In [146]:
dc_21

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
16,32645338,NaN,2021-01-01 01:53:35,2021-01-01 01:53:35,0.25,Cash,IPS,23408240
17,32645338,NaN,2021-01-01 01:53:42,2021-01-01 01:53:42,0.25,Cash,IPS,23408241
18,32645338,NaN,2021-01-01 01:53:44,2021-01-01 01:53:44,0.25,Cash,IPS,23408246
19,32645338,NaN,2021-01-01 01:53:46,2021-01-01 01:53:46,0.25,Cash,IPS,23408245
20,32645338,NaN,2021-01-01 01:54:46,2021-01-01 01:54:46,0.25,Cash,IPS,23408242
...,...,...,...,...,...,...,...,...
6748137,12122808,NaN,2021-12-30 23:31:11,2021-12-30 23:31:11,0.25,Cash,IPS,24834153
6748138,12122808,NaN,2021-12-30 23:31:13,2021-12-30 23:31:13,0.25,Cash,IPS,24834152
6748139,12122808,NaN,2021-12-30 23:31:15,2021-12-30 23:31:15,0.25,Cash,IPS,24834154
6748140,12122808,NaN,2021-12-30 23:31:17,2021-12-30 23:31:17,0.25,Cash,IPS,24834155


In [147]:
# Checking if 'transmissionid' is a unique key
transmission_counts = dc_21['transmissionid'].value_counts()

# Filtering transmission IDs to only show duplicates
duplicate_transmissions = transmission_counts[transmission_counts > 1]

In [148]:
duplicate_transmissions

1663337356066452200    2
1609671890074061300    2
1610645437007565800    2
1612272072010202200    2
24679927               2
1637764241052449900    2
1611447203088978700    2
1626628681015949000    2
1675264064012215000    2
Name: transmissionid, dtype: int64

In [149]:
# It appears that certain transmission ID's are repeated if they were sent on the same day at the same meter 
# and were paid for using cash or card, therefore will add in payment start and expiration to help 
# differentiate when removing duplicates
# Examining specific instance here:
verification_rows = dc_21[dc_21['transmissionid'] == '1675264064012215000']
verification_rows

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
1943216,82732791,NaN,2021-06-19 20:06:47,2021-06-19 22:59:58,5.77,CreditCard,Parkeon,1675264064012215000
2812798,82732791,NaN,2021-07-24 16:07:14,2021-07-24 18:07:14,4.00,CreditCard,Parkeon,1675264064012215000


In [150]:
# Loading 2022 data
dc_trans_22 = pd.read_parquet('dc_2022_parking_payments.parquet')

In [151]:
dc_trans_22

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
0,NULL,22057,2022-01-01 00:01:50,2022-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,917a1c5f-ceea-4b46-9ba5-770520859d2e
1,NULL,22866,2022-01-01 00:02:20,2022-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,126a53cf-7e8b-4c8f-a0ef-20ec862b81fb
2,NULL,22091,2022-01-01 00:02:57,2022-01-01 01:02:57,0.00,PayByCell,PARKMOBILE,df746cd0-fe13-4941-a9d9-c7f88f171e99
3,NULL,22307,2022-01-01 00:03:07,2022-01-01 02:03:07,0.00,PayByCell,PARKMOBILE,80e5aae8-b2db-4062-a661-e226bec9b65a
4,NULL,21792,2022-01-01 00:04:33,2022-01-01 23:59:59,0.00,PayByCell,PARKMOBILE,6b3e513a-bf59-4e34-baac-a2331fef50c0
...,...,...,...,...,...,...,...,...
9091622,21031802,NULL,2022-12-31 23:51:48,2022-12-31 23:51:48,0.25,Cash,IPS,26543140
9091623,21031802,NULL,2022-12-31 23:52:00,2022-12-31 23:52:00,0.25,Cash,IPS,26543141
9091624,NULL,21725,2022-12-31 23:52:25,2022-12-31 23:59:59,0.00,PayByCell,PARKMOBILE,89222d4c-5e12-4616-8fe3-19a76388e5e4
9091625,NULL,24541,2022-12-31 23:56:41,2022-12-31 23:59:59,0.15,PayByCell,PARKMOBILE,4a84aca7-3407-4473-8196-1b331f987124


In [152]:
# 2022 null count to see how many instances we will lose due to lack of geo data 
# rather than a 0 payment or repeat data
dc_22 = renaming_nulls(dc_trans_22)
null_count_22 = dc_22[(dc_22['meternumber'].isnull()) & (dc_22['zone'].isnull())].shape[0]
null_count_22

0

In [153]:
# More cleaning for uniformity
dc_22 = dc_22.dropna(subset=['meternumber', 'zone'], how='all')
dc_22 = dc_22.drop(dc_22[dc_22['paymentamount'] == 0.00].index)
dc_22 = dc_22.drop_duplicates()

In [154]:
dc_22

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
13,20181313,NaN,2022-01-01 00:13:25,2022-01-01 00:13:25,0.25,Cash,IPS,24837571
14,20181313,NaN,2022-01-01 00:13:26,2022-01-01 00:13:26,0.25,Cash,IPS,24837572
15,20181313,NaN,2022-01-01 00:13:29,2022-01-01 00:13:29,0.25,Cash,IPS,24837577
16,20181313,NaN,2022-01-01 00:13:31,2022-01-01 00:13:31,0.25,Cash,IPS,24837575
17,20181313,NaN,2022-01-01 00:13:34,2022-01-01 00:13:34,0.05,Cash,IPS,24837576
...,...,...,...,...,...,...,...,...
9091609,63010806,NaN,2022-12-31 23:31:25,2022-12-31 23:31:25,0.25,Cash,IPS,26543176
9091610,63010806,NaN,2022-12-31 23:31:34,2022-12-31 23:31:34,0.25,Cash,IPS,26543177
9091622,21031802,NaN,2022-12-31 23:51:48,2022-12-31 23:51:48,0.25,Cash,IPS,26543140
9091623,21031802,NaN,2022-12-31 23:52:00,2022-12-31 23:52:00,0.25,Cash,IPS,26543141


In [155]:
# Null counts
dc_22.isna().sum()

meternumber          5484354
zone                 3373297
paymentstart               0
paymentexpiry              0
paymentamount              0
paymenttype                0
transactionsender          0
transmissionid             0
dtype: int64

In [156]:
# Loading 2023 data
dc_trans_23 = pd.read_parquet('dc_2023_parking_payments.parquet')

In [157]:
# 2023 null count to see how many instances we will lose due to lack of geo data 
# rather than a 0 payment or repeat data
dc_23 = renaming_nulls(dc_trans_23)
null_count_23 = dc_23[(dc_23['meternumber'].isnull()) & (dc_23['zone'].isnull())].shape[0]
null_count_23

0

In [158]:
# More cleaning for uniformity
dc_23 = dc_23.dropna(subset=['meternumber', 'zone'], how='all')
dc_23 = dc_23.drop(dc_23[dc_23['paymentamount'] == 0.00].index)
dc_23 = dc_23.drop_duplicates()

In [159]:
dc_23

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid
21,14872508,NaN,2023-01-01 00:29:01,2023-01-01 00:29:01,0.25,Cash,IPS,26543505
22,14872508,NaN,2023-01-01 00:29:04,2023-01-01 00:29:04,0.25,Cash,IPS,26543506
23,14872508,NaN,2023-01-01 00:29:06,2023-01-01 00:29:06,0.25,Cash,IPS,26543509
24,14872508,NaN,2023-01-01 00:29:08,2023-01-01 00:29:08,0.25,Cash,IPS,26543507
25,14872508,NaN,2023-01-01 00:29:11,2023-01-01 00:29:11,0.25,Cash,IPS,26543510
...,...,...,...,...,...,...,...,...
4853828,60631292,NaN,2023-06-30 11:25:41,2023-06-30 11:49:41,0.92,CreditCard,Parkeon,1688128014064435100
4853829,60631292,NaN,2023-06-30 11:55:46,2023-06-30 12:07:46,0.46,CreditCard,Parkeon,1688129824087148500
4853830,21060991,NaN,2023-06-30 20:21:34,2023-06-30 22:59:59,6.07,CreditCard,Parkeon,1688124186029183400
4853831,21060991,NaN,2023-06-30 20:22:18,2023-06-30 22:59:59,6.04,CreditCard,Parkeon,1688124228042720800


In [160]:
# Null counts
dc_23.isna().sum()

meternumber          2919589
zone                 1817754
paymentstart               0
paymentexpiry              0
paymentamount              0
paymenttype                0
transactionsender          0
transmissionid             0
dtype: int64

In [161]:
# Loading in zone and meter data from DDOT that contains geospatial identifying data 
dc_zones = pd.read_parquet('dc_meters_and_zones.parquet')

In [162]:
# In "dc_zones," columns have an unnecessary ".0" following their entries, converting to integers to remove this
dc_zones['meter_id'] = dc_zones['meter_id'].astype(int)
dc_zones['g_parkmobi'] = dc_zones['g_parkmobi'].astype(int) 
dc_zones['numberofsp'] = dc_zones['numberofsp'].astype(int)

In [163]:
# Dropping duplicates and entries where zone is listed as 0, which is not a zone in DC but a common data entry
dc_zones = dc_zones.drop_duplicates().copy()
dc_zones = dc_zones[dc_zones['g_parkmobi'] != 0]

In [164]:
# Creating new zone table with only unique addresses to perform zone merge on
dc_zones_unique = dc_zones.dropna(subset=['g_parkmobi'])
# For zones, creating a total number of spaces across all meters in a zone (that we know of) to get a more accurate denominator
dc_zones_unique['total_numberofsp'] = dc_zones_unique.groupby('g_parkmobi')['numberofsp'].transform('sum')
dc_zones_unique.drop(columns=['numberofsp'], inplace=True)
dc_zones_unique.rename(columns={'total_numberofsp': 'numberofsp'}, inplace=True)
# Going down to one zone to prevent many-to-one matching. Meter number will likely be wrong but the space number captured is our best estimate
dc_zones_unique = dc_zones_unique.drop_duplicates(subset=['g_parkmobi']).copy()
dc_zones_unique['g_parkmobi'] = pd.to_numeric(dc_zones_unique['g_parkmobi'], errors='coerce')

In [165]:
# Performing Joins
# Setting up base zones and checking categories for all joins
# Converting join columns to strings for match
na_check = ['zone', 'meternumber']
dc_zones_unique['g_parkmobi'] = dc_zones_unique['g_parkmobi'].astype(str)
dc_zones['meter_id'] = dc_zones['meter_id'].astype(str)

In [166]:
# Join for 2020: 
dc_20['zone'] = dc_20['zone'].astype(str)
dc_20['meternumber'] = dc_20['meternumber'].astype(str)

In [167]:
# First isolating values that have both meter and zone present, as they should be matched on meter for 
# highest geographic accuracy
dc_20_zone_and_meter = dc_20.dropna(subset=na_check, how="all", inplace=True)
# Removing all entries without zone
dc_20_zone_only = dc_20.dropna(subset=['zone'])
# Removing all entries without meternumber
dc_20_meter_only = dc_20.dropna(subset=['meternumber'])

In [168]:
dc_20_zone_and_meter
# Returns nothing, therefore removing from later work for 2020 as there are no
# cases of data having both a `meternumber` and a `zone`

In [169]:
# Merging on zone for 2020 data
dc_20_zones = pd.merge(dc_20_zone_only, dc_zones_unique, left_on='zone', right_on='g_parkmobi', how='inner')
# Merging on meter for 2020 meter ONLY data
dc_20_meters = pd.merge(dc_20_meter_only, dc_zones, left_on='meternumber', right_on='meter_id', how='inner')

In [170]:
# Combining the merge
full_merge_20 = pd.concat([dc_20_meters, dc_20_zones])

In [171]:
# Dropping repeats on columns discovered as identifiers when combined in earlier duplicate 'transmissionid' check
full_merge_unique_20 = full_merge_20.drop_duplicates(subset=['paymentstart', 'paymentexpiry', 'transmissionid'])

In [172]:
full_merge_unique_20

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid,meter_id,g_zone,g_parkmobi,blk_no,street,address,numberofsp,longitude,latitude
0,20151005,nan,2020-01-01 00:44:26,2020-01-01 00:44:26,1.75,Cash,IPS,21977521,20151005,208.0,22198,1000.0,15TH ST NW,1000 15TH ST NW E,1,-77.03450,38.90269
1,20151005,nan,2020-01-01 13:19:42,2020-01-01 14:17:06,2.20,Cash,IPS,21979841,20151005,208.0,22198,1000.0,15TH ST NW,1000 15TH ST NW E,1,-77.03450,38.90269
2,20151005,nan,2020-01-02 09:30:51,2020-01-02 11:30:51,4.60,CreditCard,IPS,21982754,20151005,208.0,22198,1000.0,15TH ST NW,1000 15TH ST NW E,1,-77.03450,38.90269
3,20151005,nan,2020-01-02 12:56:21,2020-01-02 14:56:21,4.60,CreditCard,IPS,21985640,20151005,208.0,22198,1000.0,15TH ST NW,1000 15TH ST NW E,1,-77.03450,38.90269
4,20151005,nan,2020-01-02 17:50:02,2020-01-02 18:03:05,0.50,Cash,IPS,21989706,20151005,208.0,22198,1000.0,15TH ST NW,1000 15TH ST NW E,1,-77.03450,38.90269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250837,nan,22746,2020-12-23 14:21:03,2020-12-23 15:21:03,2.30,PayByCell,PARKMOBILE,fb93ec7c-0366-4af1-b835-0c2cc60de1b5,42127791,911.0,22746,7700.0,GEORGIA AVE NW,7700 GEORGIA AVE NW E,10,-77.02647,38.98322
250838,nan,22746,2020-12-28 11:10:29,2020-12-28 11:17:29,0.30,PayByCell,PARKMOBILE,87de1814-0798-455f-b4b2-f74a2525a043,42127791,911.0,22746,7700.0,GEORGIA AVE NW,7700 GEORGIA AVE NW E,10,-77.02647,38.98322
250839,nan,23231,2020-12-24 09:54:40,2020-12-24 11:54:40,4.60,PayByCell,PARKMOBILE,fa7273b5-fd0c-4297-a91c-6b0b72f52fd9,20150701,208.0,23231,700.0,15TH ST NW,700 15TH ST NW E,1,-77.03374,38.89890
250840,nan,23063,2020-12-28 13:53:58,2020-12-28 14:53:58,2.30,PayByCell,PARKMOBILE,783f1eb6-3368-46a3-9c7f-0aa472b46dea,21080602,224.0,23063,600.0,H ST NW,600 H ST NW S,1,-77.02049,38.89978


In [173]:
# Join for 2021:
dc_21['zone'] = dc_21['zone'].astype(str)
dc_21['meternumber'] = dc_21['meternumber'].astype(str)

In [174]:
# First isolating values that have both meter and zone present, as they should be matched on meter for 
# highest geographic accuracy
dc_21_zone_and_meter = dc_21.dropna(subset=na_check, inplace=True)
# Removing all entries without zone
dc_21_zone_only = dc_21.dropna(subset=['zone'])
# Removing all entries without meternumber
dc_21_meter_only = dc_21.dropna(subset=['meternumber'])

In [175]:
dc_21_zone_and_meter
# Returns nothing, therefore removing from later work for 2021 as there are no
# cases of data having both a `meternumber` and a `zone`

In [176]:
# Merging on zone for 2021 data
dc_21_zones = pd.merge(dc_21_zone_only, dc_zones_unique, left_on='zone', right_on='g_parkmobi', how='inner')
# Merging on meter for 2021 meter ONLY data
dc_21_meters = pd.merge(dc_21_meter_only, dc_zones, left_on='meternumber', right_on='meter_id', how='inner')

In [177]:
# Combining the merge
full_merge_21 = pd.concat([dc_21_meters, dc_21_zones])

In [178]:
# Dropping repeats on columns discovered as identifiers when combined in earlier duplicate 'transmissionid' check
full_merge_unique_21 = full_merge_21.drop_duplicates(subset=['paymentstart', 'paymentexpiry', 'transmissionid'])

In [179]:
full_merge_unique_21

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid,meter_id,g_zone,g_parkmobi,blk_no,street,address,numberofsp,longitude,latitude
0,32645338,nan,2021-01-01 01:53:35,2021-01-01 01:53:35,0.25,Cash,IPS,23408240,32645338,306.0,21867,5300.0,WISCONSIN AVE NW,5300 WISCONSIN AVE NW W,1,-77.08553,38.96012
1,32645338,nan,2021-01-01 01:53:42,2021-01-01 01:53:42,0.25,Cash,IPS,23408241,32645338,306.0,21867,5300.0,WISCONSIN AVE NW,5300 WISCONSIN AVE NW W,1,-77.08553,38.96012
2,32645338,nan,2021-01-01 01:53:44,2021-01-01 01:53:44,0.25,Cash,IPS,23408246,32645338,306.0,21867,5300.0,WISCONSIN AVE NW,5300 WISCONSIN AVE NW W,1,-77.08553,38.96012
3,32645338,nan,2021-01-01 01:53:46,2021-01-01 01:53:46,0.25,Cash,IPS,23408245,32645338,306.0,21867,5300.0,WISCONSIN AVE NW,5300 WISCONSIN AVE NW W,1,-77.08553,38.96012
4,32645338,nan,2021-01-01 01:54:46,2021-01-01 01:54:46,0.25,Cash,IPS,23408242,32645338,306.0,21867,5300.0,WISCONSIN AVE NW,5300 WISCONSIN AVE NW W,1,-77.08553,38.96012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194110,nan,23804,2021-10-07 17:22:02,2021-10-07 17:52:02,1.15,PayByCell,PARKMOBILE,9ea88020-b7cf-49bb-87e4-8ded6ec0f3b6,20110801,237.0,23804,800.0,11TH ST NW,800 11TH ST NW E,1,-77.02697,38.90044
3194111,nan,23195,2021-10-14 18:10:26,2021-10-14 19:40:26,3.45,PayByCell,PARKMOBILE,d8f4ac3c-14e8-4c40-9644-f75102b6f59d,20120801,239.0,23195,800.0,12TH ST NW,800 12TH ST NW E,1,-77.02800,38.90000
3194112,nan,23505,2021-10-16 14:15:35,2021-10-16 15:15:35,2.30,PayByCell,PARKMOBILE,a03f15d2-932a-416e-9142-082aa5e57fd5,21101201,225.0,23505,1200.0,K ST NW,1200 K ST NW N,1,-77.02918,38.90260
3194113,nan,23505,2021-12-13 09:08:39,2021-12-13 11:08:39,4.60,PayByCell,PARKMOBILE,a2871750-ca03-4079-be2f-1b8fbe66af07,21101201,225.0,23505,1200.0,K ST NW,1200 K ST NW N,1,-77.02918,38.90260


In [180]:
# Join for 2022:
dc_22['zone'] = dc_22['zone'].astype(str)
dc_22['meternumber'] = dc_22['meternumber'].astype(str)

In [181]:
# First isolating values that have both meter and zone present, as they should be matched on meter for 
# highest geographic accuracy
dc_22_zone_and_meter = dc_22.dropna(subset=na_check, inplace=True)
# Removing all entries without zone
dc_22_zone_only = dc_22.dropna(subset=['zone'])
# Removing all entries without meternumber
dc_22_meter_only = dc_22.dropna(subset=['meternumber'])

In [182]:
dc_22_zone_and_meter # No entries present

In [183]:
# Merging on zone for 2022 data
dc_22_zones = pd.merge(dc_22_zone_only, dc_zones_unique, left_on='zone', right_on='g_parkmobi', how='inner')
# Merging on meter for 2022 meter ONLY data
dc_22_meters = pd.merge(dc_22_meter_only, dc_zones, left_on='meternumber', right_on='meter_id', how='inner')

In [184]:
# Combining the merge
full_merge_22 = pd.concat([dc_22_meters, dc_22_zones])

In [185]:
# Dropping repeats on columns discovered as identifiers when combined in earlier duplicate 'transmissionid' check
full_merge_unique_22 = full_merge_22.drop_duplicates(subset=['paymentstart', 'paymentexpiry', 'transmissionid'])

In [186]:
full_merge_unique_22

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid,meter_id,g_zone,g_parkmobi,blk_no,street,address,numberofsp,longitude,latitude
0,12123606,nan,2022-01-01 00:18:01,2022-01-01 00:21:13,0.32,Cash,IPS,24880383,12123606,102.0,22695,3600.0,GEORGIA AVE NW,3600 GEORGIA AVE NW W,1,-77.02425,38.93553
1,12123606,nan,2022-01-05 14:34:41,2022-01-05 14:47:44,0.50,Cash,IPS,24841894,12123606,102.0,22695,3600.0,GEORGIA AVE NW,3600 GEORGIA AVE NW W,1,-77.02425,38.93553
2,12123606,nan,2022-01-05 16:59:22,2022-01-05 17:08:30,0.35,Cash,IPS,24842345,12123606,102.0,22695,3600.0,GEORGIA AVE NW,3600 GEORGIA AVE NW W,1,-77.02425,38.93553
3,12123606,nan,2022-01-06 12:46:52,2022-01-06 13:19:29,1.25,Cash,IPS,24844463,12123606,102.0,22695,3600.0,GEORGIA AVE NW,3600 GEORGIA AVE NW W,1,-77.02425,38.93553
4,12123606,nan,2022-01-06 12:47:17,2022-01-06 13:39:03,0.75,Cash,IPS,24844464,12123606,102.0,22695,3600.0,GEORGIA AVE NW,3600 GEORGIA AVE NW W,1,-77.02425,38.93553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4533435,nan,23889,2022-11-29 11:01:33,2022-11-29 11:46:33,1.75,PayByCell,PARKMOBILE,6f1a8509-7e8b-40f7-8d30-a89373fd6117,23020221,242.0,23889,200.0,CONSTITUTION AVE NW,200 CONSTITUTION AVE NW N,1,-77.01495,38.89215
4533436,nan,23118,2022-12-01 09:37:41,2022-12-01 13:37:41,9.20,PayByCell,PARKMOBILE,b53d574d-401b-42b6-82cf-ef8b9460a008,60820226,228.0,23118,200.0,12TH ST SW,200 12TH ST SW E,1,-77.02801,38.88625
4533437,nan,23118,2022-12-01 13:50:51,2022-12-01 15:59:59,5.00,PayByCell,PARKMOBILE,291b429f-8f99-46a0-a5b4-e7702eed7287,60820226,228.0,23118,200.0,12TH ST SW,200 12TH ST SW E,1,-77.02801,38.88625
4533438,nan,23031,2022-12-29 16:14:09,2022-12-29 16:33:09,0.75,PayByCell,PARKMOBILE,380b373c-a04c-4403-80d2-b2f711fdd9e3,20100601,206.0,23031,600.0,10TH ST NW,600 10TH ST NW E,1,-77.02591,38.89781


In [187]:
# Join for 2023:
dc_23['zone'] = dc_23['zone'].astype(str)
dc_23['meternumber'] = dc_23['meternumber'].astype(str)

In [188]:
# First isolating values that have both meter and zone present, as they should be matched on meter for 
# highest geographic accuracy
dc_23_zone_and_meter = dc_23.dropna(subset=na_check, inplace=True)
# Removing all entries without zone
dc_23_zone_only = dc_23.dropna(subset=['zone'])
# Removing all entries without meternumber
dc_23_meter_only = dc_23.dropna(subset=['meternumber'])

In [189]:
dc_23_zone_and_meter
# Returns nothing, therefore removing from later work for 2020 as there are no
# cases of data having both a `meternumber` and a `zone`

In [190]:
# Merging on zone for 2023 data
dc_23_zones = pd.merge(dc_23_zone_only, dc_zones_unique, left_on='zone', right_on='g_parkmobi', how='inner')
# Merging on meter for 2023 data
dc_23_meters = pd.merge(dc_23_meter_only, dc_zones, left_on='meternumber', right_on='meter_id', how='inner')

In [191]:
# Combining the merge
full_merge_23 = pd.concat([dc_23_meters, dc_23_zones])

In [192]:
# Dropping repeats on columns discovered as identifiers when combined in earlier duplicate 'transmissionid' check
full_merge_unique_23 = full_merge_23.drop_duplicates(subset=['paymentstart', 'paymentexpiry', 'transmissionid'])

In [193]:
full_merge_unique_23

,meternumber,zone,paymentstart,paymentexpiry,paymentamount,paymenttype,transactionsender,transmissionid,meter_id,g_zone,g_parkmobi,blk_no,street,address,numberofsp,longitude,latitude
0,14872508,nan,2023-01-01 00:29:01,2023-01-01 00:29:01,0.25,Cash,IPS,26543505,14872508,104.0,21641,2500.0,ONTARIO RD NW,2500 ONTARIO RD NW W,1,-77.04035,38.92398
1,14872508,nan,2023-01-01 00:29:04,2023-01-01 00:29:04,0.25,Cash,IPS,26543506,14872508,104.0,21641,2500.0,ONTARIO RD NW,2500 ONTARIO RD NW W,1,-77.04035,38.92398
2,14872508,nan,2023-01-01 00:29:06,2023-01-01 00:29:06,0.25,Cash,IPS,26543509,14872508,104.0,21641,2500.0,ONTARIO RD NW,2500 ONTARIO RD NW W,1,-77.04035,38.92398
3,14872508,nan,2023-01-01 00:29:08,2023-01-01 00:29:08,0.25,Cash,IPS,26543507,14872508,104.0,21641,2500.0,ONTARIO RD NW,2500 ONTARIO RD NW W,1,-77.04035,38.92398
4,14872508,nan,2023-01-01 00:29:11,2023-01-01 00:29:11,0.25,Cash,IPS,26543510,14872508,104.0,21641,2500.0,ONTARIO RD NW,2500 ONTARIO RD NW W,1,-77.04035,38.92398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468049,nan,23803,2023-06-28 17:39:42,2023-06-28 18:39:42,2.30,PayByCell,PARKMOBILE,bac6b239-085e-47e8-95e4-8c598035ff16,20101002,237.0,23803,1000.0,10TH ST NW,1000 10TH ST NW W,1,-77.02608,38.90313
2468050,nan,23803,2023-06-28 18:40:40,2023-06-28 19:40:40,2.30,PayByCell,PARKMOBILE,f9cf1f23-134a-426f-b743-f0b0ad0a4eac,20101002,237.0,23803,1000.0,10TH ST NW,1000 10TH ST NW W,1,-77.02608,38.90313
2468051,nan,23523,2023-06-29 10:37:28,2023-06-29 12:37:28,6.75,PayByCell,PARKMOBILE,24e59a23-cee1-4a46-8a94-a32551d096cd,21070701,225.0,23523,700.0,G ST NW,700 G ST NW N,1,-77.02275,38.89853
2468052,nan,23523,2023-06-29 12:32:36,2023-06-29 14:37:28,9.50,PayByCell,PARKMOBILE,dbe61dce-a96b-4f03-9a66-5a9b85bb8523,21070701,225.0,23523,700.0,G ST NW,700 G ST NW N,1,-77.02275,38.89853


In [194]:
# For transactions including `meter_id`, `zone` is missing because it was either payed
# by card or cash, therefore am shifting the `g_parkmobile` zone value merged from 
# `dc_zones.parquet` to create a more complete zone column, and shifting `meternumber` to 
# create a more complete meter column

full_merge_unique_20_copy = full_merge_unique_20.copy()
# NaN's are appearing as "nan" strings, resetting here
full_merge_unique_20_copy['zone'] = full_merge_unique_20_copy['zone'].replace('nan', np.nan)
full_merge_unique_20_copy['meternumber'] = full_merge_unique_20_copy['meternumber'].replace('nan', np.nan)
#Filling missing values
full_merge_unique_20_copy['zone'] = full_merge_unique_20_copy['zone'].fillna(full_merge_unique_20_copy['g_parkmobi']).replace(['0', 0], np.nan)
full_merge_unique_20_copy['meternumber'] = full_merge_unique_20_copy['meternumber'].fillna(full_merge_unique_20_copy['meter_id'])

full_merge_unique_21_copy = full_merge_unique_21.copy()
# Replacing nans
full_merge_unique_21_copy['zone'] = full_merge_unique_21_copy['zone'].replace('nan', np.nan)
full_merge_unique_21_copy['meternumber'] = full_merge_unique_21_copy['meternumber'].replace('nan', np.nan)
#Filling missing values
full_merge_unique_21_copy['zone'] = full_merge_unique_21_copy['zone'].fillna(full_merge_unique_21_copy['g_parkmobi']).replace(['0', 0], np.nan)
full_merge_unique_21_copy['meternumber'] = full_merge_unique_21_copy['meternumber'].fillna(full_merge_unique_21_copy['meter_id'])

full_merge_unique_22_copy = full_merge_unique_22.copy()
# Replacing nans
full_merge_unique_22_copy['zone'] = full_merge_unique_22_copy['zone'].replace('nan', np.nan)
full_merge_unique_22_copy['meternumber'] = full_merge_unique_22_copy['meternumber'].replace('nan', np.nan)
#Filling missing values
full_merge_unique_22_copy['zone'] = full_merge_unique_22_copy['zone'].fillna(full_merge_unique_22_copy['g_parkmobi']).replace(['0', 0], np.nan)
full_merge_unique_22_copy['meternumber'] = full_merge_unique_22_copy['meternumber'].fillna(full_merge_unique_22_copy['meter_id'])

full_merge_unique_23_copy = full_merge_unique_23.copy()
# Replacing nans
full_merge_unique_23_copy['zone'] = full_merge_unique_23_copy['zone'].replace('nan', np.nan)
full_merge_unique_23_copy['meternumber'] = full_merge_unique_23_copy['meternumber'].replace('nan', np.nan)
#Filling missing values
full_merge_unique_23_copy['zone'] = full_merge_unique_23_copy['zone'].fillna(full_merge_unique_23_copy['g_parkmobi']).replace(['0', 0], np.nan)
full_merge_unique_23_copy['meternumber'] = full_merge_unique_23_copy['meternumber'].fillna(full_merge_unique_23_copy['meter_id'])

In [195]:
# Dropping excess columns
def drop_repeat_cols(df):
    return df.drop(columns=['transmissionid', 'meter_id', 'g_zone', 'g_parkmobi'])

In [196]:
merge_20 = drop_repeat_cols(full_merge_unique_20_copy)
merge_21 = drop_repeat_cols(full_merge_unique_21_copy)
merge_22 = drop_repeat_cols(full_merge_unique_22_copy)
merge_23 = drop_repeat_cols(full_merge_unique_23_copy)

In [197]:
# Cleaning spare .0 on block numbers
merge_20['blk_no'] = merge_20['blk_no'].astype(int)
merge_21['blk_no'] = merge_21['blk_no'].astype(int)
merge_22['blk_no'] = merge_22['blk_no'].astype(int)
merge_23['blk_no'] = merge_23['blk_no'].astype(int)

In [198]:
# Converting everything to string due to dtype issue with pyarrow that was not allowing files to be saved

In [199]:
def to_string_converter(df):
    df_str = df.astype(str)
    return df_str

In [200]:
merge_20 = to_string_converter(merge_20)
merge_21 = to_string_converter(merge_21)
merge_22 = to_string_converter(merge_22)
merge_23 = to_string_converter(merge_23)

In [201]:
# Saving files:

In [202]:
# Defining path for the Parquet file for 2021 merge
merge_20_file = "dc_merge_20.parquet"

# Writing the DataFrame to a Parquet file
full_merge_20 = pa.Table.from_pandas(merge_20)
pq.write_table(full_merge_20, merge_20_file)

In [203]:
# Defining path for the Parquet file for 2021 merge
merge_21_file = "dc_merge_21.parquet"

# Writing the DataFrame to a Parquet file
full_merge_21 = pa.Table.from_pandas(merge_21)
pq.write_table(full_merge_21, merge_21_file)

In [204]:
# Defining path for the Parquet file for 2022 merge
merge_22_file = "dc_merge_22.parquet"

# Writing the DataFrame to a Parquet file
full_merge_22 = pa.Table.from_pandas(merge_22)
pq.write_table(full_merge_22, merge_22_file)

In [205]:
# Defining path for the Parquet file for 2023 merge
merge_23_file = "dc_merge_23.parquet"

# Writing the DataFrame to a Parquet file
full_merge_23 = pa.Table.from_pandas(merge_23)
pq.write_table(full_merge_23, merge_23_file)